#### **1. Import dữ liệu đầu vào**

In [ ]:
from Crypto.Util.number import *
from secret import n, xG, yG
import ast

#### **2. Class `DummyPoint`**
- Mô phỏng điểm trên đường cong elliptic tùy chỉnh

##### ***2.1. `__init__(self, x=None, y=None)`***
- Nếu không truyền `x`, `y` → Điểm vô cực (`infinity`).
- Ngược lại, khởi tạo điểm bình thường, đảm bảo tọa độ nằm trên đường cong (phương thức `isOnCurve`).

##### ***2.2. `@classmethod infinity`***
- Tạo điểm vô cực `𝑂`

##### ***2.3. `is_infinity(self)`***
- Kiểm tra xem điểm đó có phải vô cực hay không?

##### ***2.4. `isOnCurve(x, y)`***
- Xác định điểm có nằm trên đường cong hay không → `nhưng đã bị che giấu` nên ta không biết phương trình đường cong.

##### ***2.5. `__add__`***
- ECC định nghĩa phép cộng $𝑃+𝑄$, có 2 trường hợp chính. 
- Nếu một trong hai điểm là điểm vô cực -> Trả về điểm còn lại
- TH1. $P \neq Q$
    - Tính slope $s$ theo công thức: 
    $$s = \frac{y_1-y_2}{x_1-x_2} \pmod n$$

    - Tính nghịch đảo lũy thừa bậc 3 của slope.
    $$s_3 = (\frac{1}{s})^3 \pmod n$$
    Thông thường, chuẩn sẽ là $s^2 \pmod n$

    - Tính phần "***dị***" dính đến 1337
    $$d = y_1 - s.x_1$$

    - Tính hoành độ mới `x`:
    $$x' = (-3(d-1337).s^{-1} + (s^{-1})^3 - (x_1+x_2)) \pmod n$$

    - Tính tung độ `y`:
    $$y' = y_1 - s(x_1 - x')$$

- TH2: $P == Q$
    - Tính slope $s$ theo công thức:
    $$s = \frac{2x}{3(y-1337^2)} \pmod n$$
##### ***2.6. `__rmul__`***
- Phép nhân vô hướng. Thực hiện nhân điểm $k⋅P$ theo thuật toán double-and-add (dạng nhị phân). Đây là cách tạo các điểm ECC mới từ điểm sinh $G$.

##### ***2.7. `__repr__(self)`***
- In biểu diễn

##### ***2.8. `__eq__(self, other)`*** 
- So sánh hai điểm có bằng nhau hay không?

In [ ]:
class DummyPoint:
    O = object()

    def __init__(self, x=None, y=None):
        if (x, y) == (None, None):
            self._infinity = True
        else:
            assert DummyPoint.isOnCurve(x, y), (x, y)
            self.x, self.y = x, y
            self._infinity = False

    @classmethod
    def infinity(cls):
        return cls()

    def is_infinity(self):
        return getattr(self, "_infinity", False)

    @staticmethod
    def isOnCurve(x, y):
        return "<REDACTED>"

    def __add__(self, other):
        if other.is_infinity():
            return self
        if self.is_infinity():
            return other

        # ——— Distinct‑points case ———
        if self.x != other.x or self.y != other.y:
            dy    = self.y - other.y
            dx    = self.x - other.x
            inv_dx = pow(dx, -1, n)
            prod1 = dy * inv_dx
            s     = prod1 % n

            inv_s = pow(s, -1, n)
            s3    = pow(inv_s, 3, n)

            tmp1 = s * self.x
            d    = self.y - tmp1

            d_minus    = d - 1337
            neg_three  = -3
            tmp2       = neg_three * d_minus
            tmp3       = tmp2 * inv_s
            sum_x      = self.x + other.x
            x_temp     = tmp3 + s3
            x_pre      = x_temp - sum_x
            x          = x_pre % n

            tmp4       = self.x - x
            tmp5       = s * tmp4
            y_pre      = self.y - tmp5
            y          = y_pre % n

            return DummyPoint(x, y)

        dy_term       = self.y - 1337
        dy2           = dy_term * dy_term
        three_dy2     = 3 * dy2
        inv_3dy2      = pow(three_dy2, -1, n)
        two_x         = 2 * self.x
        prod2         = two_x * inv_3dy2
        s             = prod2 % n

        inv_s         = pow(s, -1, n)
        s3            = pow(inv_s, 3, n)

        tmp6          = s * self.x
        d2            = self.y - tmp6

        d2_minus      = d2 - 1337
        tmp7          = -3 * d2_minus
        tmp8          = tmp7 * inv_s
        x_temp2       = tmp8 + s3
        x_pre2        = x_temp2 - two_x
        x2            = x_pre2 % n

        tmp9          = self.x - x2
        tmp10         = s * tmp9
        y_pre2        = self.y - tmp10
        y2            = y_pre2 % n

        return DummyPoint(x2, y2)

    def __rmul__(self, k):
        if not isinstance(k, int) or k < 0:
            raise ValueError("Choose another k")
        
        R = DummyPoint.infinity()
        addend = self
        while k:
            if k & 1:
                R = R + addend
            addend = addend + addend
            k >>= 1
        return R

    def __repr__(self):
        return f"DummyPoint({self.x}, {self.y})"

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y